# Spark User Defined Functions

## Overview

Spark provides a powerful computational engine and we have seen some core aspects of it thus far. In this
section, we will look into how to create user defined functions and integrate these in Spark.

## User defined functions in Spark

## References

1. Jules S. Damji, Brooke Wenig, Tathagata Das, Deny Lee, _Learning Spark. Lighting-fasts data analytics_, 2nd Edition, O'Reilly.